In [2]:
import numpy as np
import os
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score,roc_curve
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [3]:
data = pd.read_csv('hmelq.csv')
data = data.dropna()

In [4]:
X = data.drop(['bad'], axis = 1)
y = data['bad']

X = pd.get_dummies(X)
X.head()

,loan,mortdue,value,yoj,derog,delinq,clage,ninq,clno,debtinc,reason_DebtCon,reason_HomeImp,job_Mgr,job_Office,job_Other,job_ProfEx,job_Sales,job_Self
153,18200,94727.0,136877.0,15.0,0.0,0.0,168.96,2.0,26.0,36.056,1,0,1,0,0,0,0,0
154,21700,79240.0,96784.0,5.0,0.0,0.0,64.51,6.0,24.0,38.079,1,0,1,0,0,0,0,0
155,34100,241931.0,36486.0,1.0,0.0,2.0,196.01,3.0,50.0,42.459,1,0,1,0,0,0,0,0
156,8400,62989.0,76718.0,3.0,0.0,2.0,131.47,0.0,22.0,29.200,0,1,1,0,0,0,0,0
157,17400,25859.0,43684.0,16.0,1.0,0.0,95.36,1.0,17.0,27.108,1,0,1,0,0,0,0,0


In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3, random_state = 42)


In [6]:
decion = DecisionTreeClassifier()
model = decion.fit(X_train,y_train)
model

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [7]:
pred = model.predict(X_test)
print(confusion_matrix(y_test,pred))
print(accuracy_score(y_test,pred))

[[538  16]
 [ 27  25]]
0.929042904290429


In [24]:
cros = cross_val_score(decion,X_train,y_train,cv= 10).mean()
cros

0.921317907444668

In [25]:
#grid search

In [26]:
cart_grid = {'max_depth': range(1,10),
            'min_samples_split': list(range(2,50))}


In [27]:
grid_decion = GridSearchCV(decion,cart_grid,cv = 10 , verbose= 2 ,n_jobs=-1)
grid_decion_model = grid_decion.fit(X_train,y_train)

Fitting 10 folds for each of 432 candidates, totalling 4320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 1899 tasks      | elapsed:    7.2s
[Parallel(n_jobs=-1)]: Done 4320 out of 4320 | elapsed:   13.2s finished


In [28]:
print(grid_decion_model.best_params_)
print('best score : {}'.format(grid_decion_model.best_score_))

{'max_depth': 8, 'min_samples_split': 2}
best score : 0.9348441926345609


In [30]:
RFC = RandomForestClassifier()
model_RFC = RFC.fit(X_train,y_train)
pred_RFC = model_RFC.predict(X_test)



C:\Users\numan\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [31]:
print(confusion_matrix(y_test,pred_RFC))
print(accuracy_score(y_test,pred_RFC))

[[553   1]
 [ 29  23]]
0.9504950495049505


In [32]:
cros = cross_val_score(RFC,X_train,y_train,cv= 10).mean()
cros

0.9412072434607645

In [34]:
RFC_params = {'max_depth': [2,5,8,10],
           'max_features': [2,5,8],
           'n_estimators': [10,500,1000],
           'min_samples_split': [2,5,10]}


In [36]:
RFC_grid = GridSearchCV(RFC,RFC_params,cv=10,n_jobs=-1,verbose=5)
model_RFC_grid = RFC_grid.fit(X_train,y_train)
model_RFC_grid

Fitting 10 folds for each of 108 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   41.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  4.3min finished


GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [2, 5, 8, 10], 'max_features': [2, 5, 8], 'n_estimators': [10, 500, 1000], 'min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=5)

In [37]:
print(model_RFC_grid.best_params_)
print('best score : {}'.format(model_RFC_grid.best_score_))

{'max_depth': 10, 'max_features': 8, 'min_samples_split': 2, 'n_estimators': 10}
best score : 0.9426345609065155


In [1]:
cat_params = {'iteration':[200,500],
             'learning_rate':[0.01,0.05,0.1],
             'depth':[3,5,8]}

In [8]:
cat = CatBoostClassifier()
model_cat = GridSearchCV(cat,cat_params,cv= 10,n_jobs=-1,verbose=5)
model_cat.fit(X_train,y_train)
model_cat

Fitting 10 folds for each of 18 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    5.6s


CatBoostError: c:/goagent/pipelines/buildmaster/catboost.gittt/catboost/libs/options/plain_options_helper.cpp:510: Unknown option {iteration} with value "200"

In [9]:
mlpc_params = {"alpha": [0.1, 0.01, 0.02, 0.005, 0.0001,0.00001],
              "hidden_layer_sizes": [(10,10,10),
                                     (100),
                                     (100,100),
                                     (3,5), 
                                     (5, 3)],
              "solver" : ["lbfgs","adam","sgd"],
              "activation": ["relu","logistic"]}

In [10]:
from sklearn.neural_network import MLPClassifier
ANN = MLPClassifier()
grid_ann = GridSearchCV(ANN,mlpc_params,cv = 10 ,n_jobs= -1,verbose = 5)
model_ann = grid_ann.fit(X_train,y_train)

Fitting 10 folds for each of 180 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done 272 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   18.6s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   25.4s
[Parallel(n_jobs=-1)]: Done 866 tasks      | elapsed:   34.0s
[Parallel(n_jobs=-1)]: Done 1136 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 1800 out of 1800 | elapsed:  3.1min finished
C:\Users\numan\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will chan

In [ ]:
print(model_ann.best_params_,'x:' , model_ann.best_score_)